In [23]:
from pyspark import SparkContext
# from pyspark.RDD import rdd

sc = SparkContext(master = "local[4]")



In [114]:
def format_result(x):
    x[1].append(x[0][0])
    return tuple(x[1])

def to_tuple(s):
    parsed_s = s.split(' ')
    res = [int(item) for item in parsed_s]
    return tuple(res)
    
def non_duplicates(s):
    a = [item for n, item in enumerate(s) if item not in s[:n]]
    return a

def get_cartesian(a):
    res = a.cartesian(a).filter(lambda x: x[0]!=x[1])
#     r = [item for item in b if item[0]!=item[1]]
    return res
    
def find_slope(t):
    if t[0][0]-t[1][0] == 0:
        k = 'inf'
    else:
        k = (t[0][1]-t[1][1])/(t[0][0]-t[1][0])
    return ((t[0], k), t[1])


def to_list(a):
    return [a]
def append(a, b):
    a.append(b)
    return a
def extend(a, b):
    a.extend(b)
    return a

def transform_to_set(l):
    return set

def find_collinear(rdd):
    a = get_cartesian(rdd)
    a_with_slope = a.map(lambda x: find_slope(x))
    res = a_with_slope.combineByKey(to_list, append, extend)
    formated = res.map(lambda x: format_result(x))
    processed = formated.filter(lambda x: len(x)>2)
    trans_set = processed.map(lambda x: set(x)).collect()
    filtered_set = [s for i, s in enumerate(trans_set) if s not in trans_set[:i]]
    final_list = [list(s) for s in filtered_set]
    print (final_list)

    return (a_with_slope)
#     return (a)

def build_collinear_set(rdd):
    mapped_rdd = rdd.map(lambda x: to_tuple(x))
    result = find_collinear(mapped_rdd)
    rdd_final = sc.parallelize(result)
    return rdd
    
def process(filename):
    rdd = sc.textFile(filename)
    rdd = build_collinear_set(rdd)
#     res = set(rdd.collect())
#     print (res)
    return rdd
  

result = process('data50.txt')
print (result.collect())
# a = [(4, -2), (2, -1), (-3,4), (6,3), (-9,4), (6, -3), (8,-4), (6,9)]
# # a = [(1,2), (2, 2), (3, 4)]
# rdd_a = sc.parallelize(a)
# res = get_cartesian(rdd_a)
# find_collinear(rdd_a)
# print (find_collinear(rdd_a).collect())    

[[(2, 0), (1, 1), (0, 2)], [(0, 7), (0, 5), (0, 4), (0, 3), (0, 8), (0, 2)], [(1, 2), (1, 3), (1, 5), (1, 8), (1, 6), (1, 0), (1, 1)], [(1, 8), (0, 8), (7, 8)], [(0, 8), (4, 6), (6, 5)], [(4, 5), (7, 8), (8, 9), (1, 2)], [(1, 5), (3, 1), (0, 7)], [(0, 8), (1, 6), (2, 4)], [(1, 2), (2, 4), (3, 6)], [(7, 3), (1, 3), (3, 3), (9, 3), (4, 3), (0, 3)], [(1, 5), (3, 3), (6, 0), (2, 4)], [(4, 5), (6, 9), (5, 7), (3, 3)], [(4, 5), (4, 6), (4, 3)], [(1, 6), (3, 3), (5, 0)], [(1, 5), (5, 7), (7, 8), (3, 6)], [(8, 9), (2, 5), (5, 7)], [(5, 9), (5, 7), (5, 2), (5, 0)], [(8, 9), (7, 6), (5, 0)], [(9, 5), (7, 6), (5, 7)], [(5, 2), (6, 0), (3, 6)], [(5, 9), (7, 6), (9, 3)], [(0, 2), (3, 1), (6, 0)], [(1, 2), (2, 0), (0, 4)], [(7, 3), (3, 1), (9, 4), (1, 0), (5, 2)], [(8, 1), (6, 1), (3, 1), (1, 1)], [(1, 2), (3, 1), (5, 0)], [(8, 9), (2, 0), (4, 3)], [(7, 4), (2, 4), (9, 4), (0, 4)], [(2, 5), (4, 6), (0, 4)], [(4, 5), (5, 9), (3, 1)], [(7, 8), (6, 5), (5, 2)], [(9, 3), (5, 2), (1, 1)], [(7, 4), (9, 3)

TypeError: 'PipelinedRDD' object is not iterable

In [43]:
# some unit tests
assert type(to_tuple('1 4')) == tuple, 'type incorrect'
# assert type(to_tuple('1 3'))[0]==int and type(to_tuple('1 3'))[1]==int, 'incorrect element type'
assert type(find_slope(((1,2),(3,4)))) == tuple, "Function must return a tuple"
assert find_slope(((1,2),(-7,-2)))[0][1] == 0.5, "Slope value should be 0.5"
assert find_slope(((1,2),(3,4))) == (((1,2),1),(3,4)), "Incorrect return value: Value obtained does not match"
assert find_slope(((1,2),(1,5))) == (((1,2),"inf"),(1,5)), "Incorrect return value: Value obtained must have slope 'inf'"
assert find_slope(((1,2),(2,5))) == (((1,2),3),(2,5)), "Incorrect return value: Value obtained does not match"

# some regular tests
s = ((1, 2), (3, 4), (1, 2), (5, 6))
print (non_duplicates(s))


b = [(1, 2), (3, 4)]
print (find_slope(b))
    
a = [(1, 2), (3, 4), (5, 6)]
rdd_a = sc.parallelize(a)
res = get_cartesian(rdd_a)
print (res)

[(1, 2), (3, 4), (5, 6)]


In [80]:
b = ((2, 2), (3, 4))
print (len(b))
if len(b)==2:
    print ('it is two')

2
it is two
